In [ ]:
import nltk


: 

## Part 1
Viterbi Algo

In [3]:
from nltk.corpus import treebank

nltk.download('treebank')

corpus = treebank.words()
print(corpus[:20])


ModuleNotFoundError: No module named 'nltk'

In [ ]:
tagged_words = treebank.tagged_words()
print(tagged_words[:40])
# show all the distinct tags
tags = set([tag for word, tag in tagged_words])
print(tags)

The tags we plan to use are 
- NN (Noun) {All forms of Noun are represented by NN}
- VB (Verb) {All forms of Verb are represented by VB}
- JJ (Adjective) {All forms of Adjective are represented by JJ}
- RB (Adverb)
- TO (to)
- DT (Determiner)
- PRP (Pronoun)
- other (all the remaining tags)

In [ ]:
dict = {'NN':0 , 'VB':1 , 'JJ':2 , 'RB':3 , 'TO':4 , 'DT':5 , 'PRP':6 ,'other':7}
def get_tag(tag):
    if tag.startswith('NN'):
        return dict['NN']
    elif tag.startswith('VB'):
        return dict['VB']
    elif tag.startswith('JJ'):
        return dict['JJ']
    elif tag.startswith('RB'):
        return dict['RB']
    elif tag.startswith('TO'):
        return dict['TO']
    elif tag.startswith('DT'):
        return dict['DT']
    elif tag.startswith('PRP'):
        return dict['PRP']
    else:
        return dict['other']

tagged_words = [(word, get_tag(tag)) for word, tag in tagged_words]
print(tagged_words[:40])

In [ ]:
import numpy as np
n_tags = 8
n_words = len(set(corpus))
word_to_index = {word: i for i, word in enumerate(set(corpus))}
emission_matrix = np.zeros((n_tags, n_words+1))
transition_matrix = np.zeros((n_tags, n_tags))

# count the occurences of each tag
tag_counts = np.zeros(n_tags)
prev_tag = None
for word, tag in tagged_words:
    tag_counts[tag] += 1
    emission_matrix[tag, word_to_index[word]] += 1
    if prev_tag is not None:
        transition_matrix[prev_tag, tag] += 1
    prev_tag = tag
    
    
# convert the counts to their log probabilities
emission_matrix = np.log(emission_matrix + 1) - np.log(tag_counts.reshape(-1, 1) + n_words)
transition_matrix = np.log(transition_matrix + 1) - np.log(tag_counts + n_tags)

print(emission_matrix)

In [ ]:
# implementing the Viterbi algorithm

def viterbi(words, emission_matrix, transition_matrix):
    n_tags, n_words = emission_matrix.shape
    n = len(words)
    scores = np.zeros((n_tags, n))
    backpointers = np.zeros((n_tags, n), dtype=int)
    
    # initialization
    scores[:, 0] = transition_matrix[-1, :] + emission_matrix[:, word_to_index[words[0]]]
    
    # Dynamic programming part 
    for i in range(1, n):
        for j in range(n_tags):
            score = scores[:, i - 1] + transition_matrix[:, j] + emission_matrix[j, word_to_index.get(words[i],n_words-1)]
            scores[j, i] = np.max(score)
            backpointers[j, i] = np.argmax(score)
    
    # retrieving the sequence of tags
    best_tag = np.argmax(scores[:, n - 1])
    tags = [best_tag]
    for i in range(n - 1, 0, -1):
        best_tag = backpointers[best_tag, i]
        tags.append(best_tag)
    
    return list(reversed(tags))

In [ ]:
str = "I want to go home"
words = str.split()
tags = viterbi(words, emission_matrix, transition_matrix)
print(list(zip(words, [list(dict.keys())[tag] for tag in tags])))

## Part 2
Vanilla Emotion Recognizer

In [1]:
from datasets import load_dataset
ds = load_dataset("dair-ai/emotion","split")
ds.shape

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# do tfidf vectorization of the dataset ds
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(ds['train']['text'])
Y = ds['train']['label']
print(X.shape)

In [ ]:
# use an svm to build a classifier for this dataset
from sklearn import svm

SVM1 = svm.SVC()
SVM1.fit(X, Y)

# test the classifier on the test set
X_test = vectorizer.transform(ds['test']['text'])
Y_pred = SVM1.predict(X_test)

# calculate the accuracy of the classifier
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ds['test']['label'], Y_pred)
print(accuracy)

# calculate the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ds['test']['label'], Y_pred)
print(cm)



## Part 3

In [4]:
# use the viterbi algorithm to tag the words in the dataset

for i in range(len(ds['train']['text'])):
    

X_train = vectorizer.transform(ds['train']['text'])
Y_train = ds['train']['label']

SVM2 = svm.SVC()
SVM2.fit(X_train, Y_train)

NameError: name 'ds' is not defined

In [ ]:
X_test = vectorizer.transform(ds['test']['text'])
Y_pred = SVM2.predict(X_test)

accuracy = accuracy_score(ds['test']['label'], Y_pred)
print(accuracy)

cm = confusion_matrix(ds['test']['label'], Y_pred)
print(cm)